## Prerequisites

1. Go to https://colab.research.google.com/
2. Upload this notebook with `File` -> `Upload Notebook` and upload this notebook

In [ ]:
!uv pip install -q syft-flwr  # should have syft-client installed as dependency

In [ ]:
import syft_client as sc

sc.__version__

## Login

In [ ]:
ds_email = input("Enter the Data Scientist's email: ")
ds_client = sc.login_ds(email=ds_email)

## Peer Addition

In [ ]:
ds_client.peers

Add first DO

In [ ]:
do1_email = input("Enter the First Data Owner's email: ")
ds_client.add_peer(do1_email)

In [ ]:
ds_client.peers

Add second DO

In [ ]:
do2_email = input("Enter the Second Data Owner's email: ")
ds_client.add_peer(do2_email)

In [ ]:
ds_client.peers

Make a list of DO emails

In [ ]:
do_emails = [peer.email for peer in ds_client.peers]

## Explore Datasets

In [ ]:
do1_datasets = ds_client.datasets.get_all(datasite=do1_email)
do1_datasets

do1_datasets[0].describe()

In [ ]:
do2_datasets = ds_client.datasets.get_all(datasite=do2_email)
do2_datasets

In [ ]:
do2_datasets[0].describe()

Append the mock paths to the list of mock datasets (needed to run simulation locally)

In [ ]:
mock_dataset_urls = [do1_datasets[0].mock_url, do2_datasets[0].mock_url]
mock_dataset_urls

## Propose FL Project

In [ ]:
from pathlib import Path

SYFT_FLWR_PROJECT_PATH = Path("../fl-diabetes-prediction")
print(f"FL project path: {SYFT_FLWR_PROJECT_PATH}")
print(f"Project exists: {SYFT_FLWR_PROJECT_PATH.exists()}")

In [ ]:
from loguru import logger

import syft_flwr

try:
    !rm -rf {SYFT_FLWR_PROJECT_PATH / "main.py"}
    logger.info(f"syft_flwr version = {syft_flwr.__version__}")
    syft_flwr.bootstrap(
        SYFT_FLWR_PROJECT_PATH, aggregator=ds_email, datasites=do_emails
    )
    logger.info("Bootstrapped project successfully ✅")
except Exception as e:
    logger.error(e)

## Run the FL Project in Simulation Mode (Optional)

In [ ]:
RUN_SIMULATION = False

In [ ]:
# clean up
!rm -rf {SYFT_FLWR_PROJECT_PATH / "fl_diabetes_prediction" / "__pycache__"}  # clean before submitting

if RUN_SIMULATION:
    print(f"running syft_flwr simulation with mock paths: {mock_dataset_urls}")
    # TODO: turn mock_dataset_urls into local paths?
    syft_flwr.run(SYFT_FLWR_PROJECT_PATH, mock_dataset_urls)

## Submit FL Project to DOs

In [ ]:
ds_client.submit_python_job(
    user=do1_email,
    code_path=SYFT_FLWR_PROJECT_PATH,  # Folder submission is currently disabled
)

In [ ]:
ds_client.submit_python_job(user=do2_email, code_path=SYFT_FLWR_PROJECT_PATH)

In [ ]:
ds_client.jobs

## DS submits code to itself

In [ ]:
job = ds_client.job.submit(
    name="fl-diabetes-prediction-ds-new",
    user_code_path=SYFT_FLWR_PROJECT_PATH,
    entrypoint="main.py",
)
job

In [ ]:
ds_client.job.get_all()

In [ ]:
ds_client.jobs[0].approve()

In [ ]:
ds_client.job.get_all()

In [ ]:
ds_client.process_approved_jobs()

In [ ]:
ds_client.jobs